In [9]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from skimage.feature import local_binary_pattern
from skimage import io, color
from math import sqrt, pi, exp

class GaussianNaiveBayes:
    def fit(self, X, y):
        self.classes = np.unique(y)
        self.mean = {}
        self.var = {}

        for c in self.classes:
            X_c = X[y == c]
            self.mean[c] = np.mean(X_c, axis=0)
            self.var[c] = np.var(X_c, axis=0)

    def predict(self, X):
        predictions = [self._predict(x) for x in X]
        return np.array(predictions)

    def _predict(self, x):
        posteriors = []

        for c in self.classes:
            prior = 1.0 / len(self.classes)  
            class_conditional = np.sum(np.log(self._pdf(c, x)))
            posterior = np.log(prior) + class_conditional
            posteriors.append(posterior)

        return self.classes[np.argmax(posteriors)]

    def _pdf(self, etiqueta, x):
        mean = self.mean[etiqueta]
        var = self.var[etiqueta]
        numerator = np.exp(-(x - mean) ** 2 / (2 * var))
        denominator = np.sqrt(2 * pi * var)
        return numerator / denominator

def caracteristicas(ruta):
    imagen = io.imread(ruta)

    # Convertir a escala de grises 
    if len(imagen.shape) == 2:
        imagen = color.gray2rgb(imagen)

    gris = color.rgb2gray(imagen)
    caracteristicas = local_binary_pattern(gris, P=8, R=1, method='uniform')
    hist, _ = np.histogram(caracteristicas.ravel(), bins=np.arange(0, 10), range=[0, 9])
    return hist

def cargarDatos(ruta):
    X, y = [], []

    for etiqueta, nombre in enumerate(os.listdir(ruta)):
        class_dir = os.path.join(ruta, nombre)
        
        for nombre_archivo in os.listdir(class_dir):
            ruta_archivo = os.path.join(class_dir, nombre_archivo)
            features = caracteristicas(ruta_archivo)
            X.append(features)
            y.append(etiqueta)

    return np.array(X), np.array(y)

# Cargar datos
rutaectory = "datos/"
X, y = cargarDatos(rutaectory)

# Dividir en conjunto de entrenamiento y prueba 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42, stratify=y)

# Crear y entrenar el modeloo
modelo = GaussianNaiveBayes()
modelo.fit(X_train, y_train)

# Realizar predicciones 
y_train_pred = modelo.predict(X_train)
y_test_pred = modelo.predict(X_test)

# Calcular la precisión 
accuracy_train = accuracy_score(y_train, y_train_pred)
accuracy_test = accuracy_score(y_test, y_test_pred)

print(f'Precisión (entrenamiento): {accuracy_train:.2f}')
conf_matrix_test = confusion_matrix(y_train, y_train_pred)
print('Matriz de confusión (entrenamiento):')
print(conf_matrix_test)

print(f'Precisión (prueba): {accuracy_test:.2f}')
conf_matrix = confusion_matrix(y_test, y_test_pred)
print('Matriz de confusión (prueba):')
print(conf_matrix)


c:\Users\chj06\AppData\Local\Programs\Python\Python311\Lib\site-packages\imageio\plugins\pillow.py:320: UserWarning: Loading 16-bit (uint16) PNG as int32 due to limitations in pillow's PNG decoder. This will be fixed in a future version of pillow which will make this warning dissapear.
  warnings.warn(
c:\Users\chj06\AppData\Local\Programs\Python\Python311\Lib\site-packages\skimage\feature\texture.py:353: UserWarning: Applying `local_binary_pattern` to floating-point images may give unexpected results when small numerical differences between adjacent pixels are present. It is recommended to use this function with images of integer dtype.
  warnings.warn(


Precisión (entrenamiento): 0.88
Matriz de confusión (entrenamiento):
[[11  0  0  0  0  0  7  0  0  6]
 [ 0 22  0  0  0  1  1  0  0  0]
 [ 0  0 23  0  1  0  0  0  0  0]
 [ 0  0  0 24  0  0  0  0  0  0]
 [ 0  0  2  0 22  0  0  0  0  0]
 [ 0  0  0  0  0 24  0  0  0  0]
 [ 7  0  0  0  0  0 17  0  0  0]
 [ 0  0  0  0  0  0  0 24  0  0]
 [ 0  0  0  0  0  0  0  0 24  0]
 [ 2  0  1  0  0  0  0  0  0 21]]
Precisión (prueba): 0.85
Matriz de confusión (prueba):
[[5 0 0 0 0 0 2 0 0 1]
 [0 8 0 0 0 0 0 0 0 0]
 [0 0 7 0 1 0 0 0 0 0]
 [0 0 0 8 0 0 0 0 0 0]
 [0 0 1 0 7 0 0 0 0 0]
 [0 0 0 0 0 8 0 0 0 0]
 [5 0 0 0 0 0 3 0 0 0]
 [0 0 0 0 0 0 0 8 0 0]
 [0 0 0 0 1 0 0 0 7 0]
 [0 0 0 0 0 0 1 0 0 7]]
